In [1]:
pwd

u'C:\\rahuketu\\programming\\Learning_C'

In [5]:
#%%writefile ipy_compiler.py

import subprocess
import os
import functools
import time
import contextlib2
import StringIO
import sys

from IPython.core.magic import register_line_magic, register_cell_magic
from IPython.display import display_html, HTML
from contextlib import contextmanager

from colorama import init
from termcolor import colored

# use Colorama to make Termcolor work on Windows too
init()

ip = get_ipython()

CC = "gcc"
CFLAGS = "-Wall -g -std=c1x"
CPATH = '"C:\Program Files (x86)\Microsoft Visual Studio 12.0\VC\include"'
CPATH = 'C:\\Anaconda\\MinGW\\x86_64-w64-mingw32\\include'
include_cpath = False

def display_command(cmd, colorfg="red", colorbg=None):
    print colored(cmd, colorfg, colorbg)

def display_ss(s):
    display_command("------%s-----" % s.upper(), colorfg="white")
    
class Shout(object):
    def __init__(self, text):
        self.text = text

    def _repr_html_(self):
        return "<h1>" + self.text + "</h1>"

@contextmanager
def stdout_redirector(stream):
    old_stdout = sys.stdout
    sys.stdout = stream
    try:
        yield
    finally:
        sys.stdout = old_stdout

def clockit(fn):
    '''Timeit function like this doesnot work with the thread calls'''
    @functools.wraps(fn)
    def wrapper(*args,**kwargs):
        display_ss("Starting %s" % fn.__name__)
        start = time.clock()
        result = fn(*args,**kwargs)
        end = time.clock()
        display_ss('Time Elapsed :%s: %.3f seconds' %( fn.__name__, round(end-start,3)))
        return result
    return wrapper

@contextlib2.contextmanager
def cd(name):
    current_dir = os.getcwd()
    if os.path.isdir(name):
        nam = name
    elif os.path.isfile(name):
        nam = os.path.dirname(os.path.abspath(name))
    else:
        raise IOError()
    os.chdir(nam)
    yield 
    os.chdir(current_dir)

@register_line_magic
@clockit
def cexy(cfile):
    with cd(cfile):
        name = os.path.basename(cfile)
        __name__ = name
        
        root = os.path.splitext(cfile)[0]
        executable = root + ".exe"
        if os.path.exists(executable):
            display_command("Removing Pre-existing %s " % executable)
            os.remove(executable)
        if include_cpath:
            runcommand = " ".join([CC, CFLAGS, cfile, "-o", root, "-I", CPATH])
        else:
            runcommand = " ".join([CC, CFLAGS, cfile, "-o", root])
    #    runcommand = ["gfortran", "--help"]
        display_command("Building executable...\n >>%s" % runcommand)
        ip.system(runcommand)
        display_command("Executing...\n >>%s" % executable)
        output = StringIO.StringIO()
        with stdout_redirector(output):
            ip.system(executable)
        display_command("\nGETTING OUTPUT\n======================================\n", "blue")
        # then use Termcolor for all colored text output
        display_command(output.getvalue(), 'blue')
#        print "\x1b[31m\"{0}\"\x1b[0m".format(output.getvalue())

@register_cell_magic
def cexy(cfile, cell):
    ip.run_cell_magic('writefile', cfile, cell)
    ip.run_line_magic('cexy', cfile)
    
"""
@register_line_magic
@clockit
def exfort(fortfile):
#    executable = ip.run_line_magic('makefort', fortfile)
    exeucutable = makefort(fortfile)
    print "Executable is : " + executable
    print "Running executable...\n >>%s" % executable
    ip.system(executable)
    
@register_cell_magic
@clockit
def mxfort(fortfile, cell):
    ip.run_cell_magic('file', fortfile, cell)
    ip.run_line_magic('exfort', fortfile)
    
@register_cell_magic
@clockit
def mbfort(fortfile, cell):
    ip.run_cell_magic('file', fortfile, cell)
    ip.run_line_magic('makefort', fortfile)
"""

with cd(os.getcwd()):
    print CC, include_cpath


ImportError: No module named contextlib2

In [39]:
%%writefile hello.c
#include <stdio.h>
#include <stdlib.h>


typedef struct{
    float x;
    float y;
} point;

int main(int argc, char **argv)
{
    point p;
    p.x = 0.2;
    p.y = 10.0;

    puts("Hello World!");
    puts("Hello World2!");
    return 0;
}

Overwriting hello.c


In [ ]:
!gcc -std=c99 -Wall -c hello.c
!gcc hello.o -o  hello

In [21]:
!hello.exe

Hello World!


In [11]:
!gcc -std=c99 -Wall hello.c -o hello

In [12]:
!hello.exe

Hello World!


In [83]:
%cexy hello.c

------STARTING CEXY-----
Removing Pre-existing hello.exe 
Building executable...
 >>gcc -Wall -g -std=c1x hello.c -o hello


hello.c: In function 'main':
hello.c:12:11: warning: variable 'p' set but not used [-Wunused-but-set-variable]


Executing...
 >>hello.exe

GETTING OUTPUT

Hello World!
Hello World2!

------TIME ELAPSED :CEXY: 0.661 SECONDS-----


In [76]:
%%writefile prompt.c

#include <stdio.h>
#include <stdlib.h>

/* If we are compiling on Windows compile these functions */
#ifdef _WIN32

#include <string.h>

static char buffer[2048];

/* Fake readline function */
char* readline(char* prompt) {p
  fputs(prompt, stdout);
  fgets(buffer, 2048, stdin);
  char* cpy = malloc(strlen(buffer)+1);
  strcpy(cpy, buffer);
  cpy[strlen(cpy)-1] = '\0';
  return cpy;
}

/* Fake add_history function */
void add_history(char* unused) {}

/* Otherwise include the editline headers */
#else

#include <editline/readline.h>
#include <editline/history.h>

#endif

int main(int argc, char** argv) {
   
  puts("Lispy Version 0.0.0.0.1");
  puts("Press Ctrl+c to Exit\n");
   
  while (1) {
    
    /* Now in either case readline will be correctly defined */
    char* input = readline("lispy> ");
    add_history(input);

    printf("No you're a %s\n", input);
    free(input);
    
  }
  
  return 0;
}

Overwriting prompt.c


In [ ]:
%cexy prompt.c

# Pointer Demo

In [52]:
%%writefile pointer_exam.c

#include <stdio.h>

int main()
{
    int var = 20; /* actual variable declaraton */
    int *ip;
    ip = &var;
    
    printf("Address of var variable: %x\n", &var); 
    printf("Address referral by using pointer ip %x\n", ip);
    printf("Value using *ip::%d\n", *ip);
    return 0;
}

Overwriting pointer_exam.c


%cexy pointer_exam.c

In [3]:
%%cexy pointer_exam2.c

#include <stdio.h>

int main()
{
    int var = 20; /* actual variable declaraton */
    int *ip;
    ip = &var;
    
    printf("Address of var variable: %x\n", &var); 
    printf("Address referral by using pointer ip %x\n", ip);
    printf("Value using *ip::%d\n", *ip);
    return 0;
}

ERROR: Cell magic `%%cexy` not found.


# flmoon :: moon phase cycle algorithm

In [ ]:
%%cexy flmoon.c

#include <math.h>
#define RAD (3.14159265/180.0)

void flmoon(int n, int nph, long *jd, float *frac)
{
    
}